In [1]:
import requests

import re
import urllib
import math
import time
import random

import pandas as pd
import sqlite3

In [2]:
my_headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Host': 'music.163.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36'
}

In [3]:
def getJSON(url, headers):
    """
    @ param url:
    @ param headers:
    @ return json:
    """
    res = requests.get(url, headers=headers)
    res.raise_for_status()  
    res.encoding = 'utf-8'  
    json = res.json()
    return json

In [4]:
def countPages(total, limit):
    """
    @ param total:
    @ param limit:
    @ return page:
    """
    page = math.ceil(total/limit) 
    return page

In [5]:
def getSongInfo(song_list):
    """
    @ param song_list:
    @ return song_info_list:
    """
    song_info_list = []
    
    for song in song_list:
        song_info = []
    
        song_info.append(song['id'])
        song_info.append(song['name'])
    
        artists_name = ''
        artists = song['artists']
        for artist in artists:
            artists_name += artist['name'] + ','
        song_info.append(artists_name)
    
        song_info.append(song['album']['name'])
        song_info.append(song['album']['id'])
        song_info.append(song['duration'])
        
        song_info_list.append(song_info)
        
    return song_info_list

In [18]:
def getSongList(key, limit=30):
    """
    @ param key:
    @ return result:
    """
    total_list = []
    key = urllib.parse.quote(key)
    url = 'http://music.163.com/api/search/get/web?csrf_token=&hlpretag=&hlposttag=&s=' + key +  '&type=1&offset=0&total=true&limit='
    
    first_page = getJSON(url, my_headers)
    song_count = first_page['result']['songCount']
    page_num = countPages(song_count, limit)
    
    for n in range(page_num):
        url = 'http://music.163.com/api/search/get/web?csrf_token=&hlpretag=&hlposttag=&s=' + key +  '&type=1&offset=' + str(n*limit) + '&total=true&limit=' + str(limit)
        tmp = getJSON(url, my_headers)
        song_list = getSongInfo(tmp['result']['songs'])
        total_list += song_list
        
        print('第 {0}/{1} 页爬取完成'.format(n+1, page_num))
        time.sleep(random.randint(2, 4)) 
        
    df = pd.DataFrame(data = total_list, columns=['song_id', 'song_name', 'artists', 'album_name', 'album_id', 'duration'])
    return df

In [7]:
def getComment(comments):
    """
        @ param comments:
        @ return comments_list:
    """
    comments_list = []
    
    for comment in comments:
        comment_info = []
        comment_info.append(comment['commentId'])
        comment_info.append(comment['user']['userId'])
        comment_info.append(comment['user']['nickname'])
        comment_info.append(comment['user']['avatarUrl'])
        comment_info.append(comment['content'])
        comment_info.append(comment['likedCount'])
        comments_list.append(comment_info)
        
    return comments_list

In [30]:
def getSongComment(id, limit=20):
    """
    @ param id:
    @ param limit:
    @ return result:
    """
    total_comment = []
    url = 'http://music.163.com/api/v1/resource/comments/R_SO_4_' + str(id) +  '?limit=20&offset=0'
    
    first_page = getJSON(url, my_headers)
    total = first_page['total']
    page_num = countPages(total, limit)
    
    for n in range(page_num):
        url = 'http://music.163.com/api/v1/resource/comments/R_SO_4_' + str(id) +  '?limit=' + str(limit) + '&offset=' + str(n*limit)
        tmp = getJSON(url, my_headers)
        comment_list = getComment(tmp['comments'])
        total_comment += comment_list
        
        print('第 {0}/{1} 页爬取完成'.format(n+1, page_num))
        time.sleep(random.randint(2, 4)) 
        
    df = pd.DataFrame(data = total_comment, columns=['comment_id', 'user_id', 'user_nickname', 'user_avatar', 'content', 'likeCount'])
    df['song_id'] = str(id)
    return df

In [9]:
conn = sqlite3.connect('netease_cloud_music.db')

In [10]:
artist='窦唯'

In [27]:
#song_df = getSongList(artist, 100)
#song_df = song_df[song_df['artists'].str.contains(artist)]
#song_df.drop_duplicates(subset=['song_id'], keep='first', inplace=True)
#song_df.to_sql(name='song', con=conn, if_exists='append', index=False)

第 1/5 页爬取完成
第 2/5 页爬取完成
第 3/5 页爬取完成
第 4/5 页爬取完成
第 5/5 页爬取完成


In [52]:
sql = '''
    SELECT song_id
    FROM song
    WHERE artists LIKE '%窦唯%'
'''

song_id = pd.read_sql(sql, con=conn)

In [53]:
song_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 1 columns):
song_id    445 non-null int64
dtypes: int64(1)
memory usage: 3.6 KB


In [54]:
song_id = song_id.loc[10:14]

In [55]:
song_id.head(10)

,song_id
10,77452
11,5282083
12,77135
13,526468453
14,77474


In [56]:
comment_df = pd.DataFrame()
for index, id in zip(song_id.index, song_id['song_id']):
    print('开始爬取第 {0}/{1} 首, {2}'.format(index+1, len(song_id['song_id']), id))
    tmp_df = getSongComment(id, 100)
    comment_df = pd.concat([comment_df, tmp_df])
comment_df.drop_duplicates(subset=['comment_id'], keep='first', inplace=True)
comment_df.to_sql(name='comment', con=conn, if_exists='append', index=False)
print('已成功保存至数据库！')

开始爬取第 11/5 首, 77452
第 1/13 页爬取完成
第 2/13 页爬取完成
第 3/13 页爬取完成
第 4/13 页爬取完成
第 5/13 页爬取完成
第 6/13 页爬取完成
第 7/13 页爬取完成
第 8/13 页爬取完成
第 9/13 页爬取完成
第 10/13 页爬取完成
第 11/13 页爬取完成
第 12/13 页爬取完成
第 13/13 页爬取完成
开始爬取第 12/5 首, 5282083
第 1/8 页爬取完成
第 2/8 页爬取完成
第 3/8 页爬取完成
第 4/8 页爬取完成
第 5/8 页爬取完成
第 6/8 页爬取完成
第 7/8 页爬取完成
第 8/8 页爬取完成
开始爬取第 13/5 首, 77135
第 1/8 页爬取完成
第 2/8 页爬取完成
第 3/8 页爬取完成
第 4/8 页爬取完成
第 5/8 页爬取完成
第 6/8 页爬取完成
第 7/8 页爬取完成
第 8/8 页爬取完成
开始爬取第 14/5 首, 526468453
第 1/94 页爬取完成
第 2/94 页爬取完成
第 3/94 页爬取完成
第 4/94 页爬取完成
第 5/94 页爬取完成
第 6/94 页爬取完成
第 7/94 页爬取完成
第 8/94 页爬取完成
第 9/94 页爬取完成
第 10/94 页爬取完成
第 11/94 页爬取完成
第 12/94 页爬取完成
第 13/94 页爬取完成
第 14/94 页爬取完成
第 15/94 页爬取完成
第 16/94 页爬取完成
第 17/94 页爬取完成
第 18/94 页爬取完成
第 19/94 页爬取完成
第 20/94 页爬取完成
第 21/94 页爬取完成
第 22/94 页爬取完成
第 23/94 页爬取完成
第 24/94 页爬取完成
第 25/94 页爬取完成
第 26/94 页爬取完成
第 27/94 页爬取完成
第 28/94 页爬取完成
第 29/94 页爬取完成
第 30/94 页爬取完成
第 31/94 页爬取完成
第 32/94 页爬取完成
第 33/94 页爬取完成
第 34/94 页爬取完成
第 35/94 页爬取完成
第 36/94 页爬取完成
第 37/94 页爬取完成
第 38/94 页爬取完成
第 39/94 页爬取完成
第 40/94 页爬取完